In [4]:
import os
import cv2
import json
import pickle
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from tqdm import tqdm
from skimage.metrics import structural_similarity as ssim

In [5]:
masked_images_folder_path = '/home/miguel/GI/0 - Data Exploration & Analysis/GI-Roberta/gi-roberta-dataset/full_dataset'

# paths = [{
#     'cv_subject': 'FD-027',
#     "styled_img_path": f"/home/miguel/GI/1.5 - Synthetic Data Generation/diffuse-gen/diffuse-gen/image_samples_FD-027/diffgen-2025-07-12-00-38/styled_samples_183x256x256x4.pkl",
# }, {
#     'cv_subject': 'FD-029',
#     "styled_img_path": f'/home/miguel/GI/1.5 - Synthetic Data Generation/diffuse-gen/diffuse-gen/image_samples_FD-029/diffgen-2025-07-12-03-48/styled_samples_185x256x256x4.pkl',
# }, {
#     'cv_subject': 'FD-030',
#     "styled_img_path": '/home/miguel/GI/1.5 - Synthetic Data Generation/diffuse-gen/diffuse-gen/image_samples_FD-030/diffgen-2025-07-12-00-38/styled_samples_189x256x256x4.pkl',
# }, {
#     'cv_subject': 'FD-031',
#     "styled_img_path": '/home/miguel/GI/1.5 - Synthetic Data Generation/diffuse-gen/diffuse-gen/image_samples_FD-031/diffgen-2025-07-12-03-56/styled_samples_180x256x256x4.pkl',
# }, {
#     'cv_subject': 'FD-032',
#     'styled_img_path': '/home/miguel/GI/1.5 - Synthetic Data Generation/diffuse-gen/diffuse-gen/image_samples_FD-032/diffgen-2025-07-12-07-05/styled_samples_183x256x256x4.pkl',
# }]

paths = [{
    'cv_subject': 'FD-027',
    'styled_img_path': '/home/miguel/GI/1.5 - Synthetic Data Generation/diffuse-gen/diffuse-gen/image_samples_FD-027/diffgen-2025-07-13-00-54/styled_samples_400x256x256x4.pkl'
}, {
    'cv_subject': 'FD-029',
    'styled_img_path': '/home/miguel/GI/1.5 - Synthetic Data Generation/diffuse-gen/diffuse-gen/image_samples_FD-029/diffgen-2025-07-13-07-47/styled_samples_560x256x256x4.pkl'
}, {
    'cv_subject': 'FD-031',
    'styled_img_path': '/home/miguel/GI/1.5 - Synthetic Data Generation/diffuse-gen/diffuse-gen/image_samples_FD-031/diffgen-2025-07-13-17-25/styled_samples_860x256x256x4.pkl'
}, {
    'cv_subject': 'FD-032',
    'styled_img_path': '/home/miguel/GI/1.5 - Synthetic Data Generation/diffuse-gen/diffuse-gen/image_samples_FD-032/diffgen-2025-07-15-02-07/styled_samples_1503x256x256x4.pkl'
}]

In [6]:
import cv2
import numpy as np

def is_cohesive_mask(mask_gray: np.ndarray,
                     thresh_method: str = 'otsu',
                     min_area: int = 50,
                     main_frac_thresh: float = 0.6,
                     max_extra_components: int = 1) -> bool:
    """
    Decide whether a grayscale mask is coherent (one big region) or incohesive
    (many small regions).

    Parameters
    ----------
    mask_gray : np.ndarray
        2D array, single-channel mask (uint8 or float in [0,1]).
    thresh_method : str
        'otsu' | 'adaptive' | 'fixed' — how to binarize.
    min_area : int
        Ignore components smaller than this (noise filter).
    main_frac_thresh : float
        Fraction of total mask area the largest component must exceed to be
        considered dominant.
    max_extra_components : int
        Allow at most this many additional (filtered) components.

    Returns
    -------
    bool
        True if “coherent” (one dominant region), False otherwise.
    """
    # ——— ensure uint8 [0,255] ———
    if mask_gray.dtype == np.float32 or mask_gray.dtype == np.float64:
        img = (mask_gray * 255).astype(np.uint8)
    else:
        img = mask_gray.astype(np.uint8)

    # ——— binarize ———
    if thresh_method == 'otsu':
        _, bw = cv2.threshold(img, 0, 255,
                              cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    elif thresh_method == 'adaptive':
        bw = cv2.adaptiveThreshold(img, 255,
                                   cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY,
                                   blockSize=11, C=2)
    else:  # 'fixed'
        _, bw = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

    # ——— connected components ———
    n_labels, labels, stats, _ = cv2.connectedComponentsWithStats(bw,
                                                                  connectivity=8)
    # stats[:, cv2.CC_STAT_AREA] is pixel-area for each label;
    # skip label 0 (background)
    areas = stats[1:, cv2.CC_STAT_AREA]

    # ——— filter out tiny noise blobs ———
    areas = areas[areas >= min_area]
    if areas.size == 0:
        return False

    total = areas.sum()
    largest = areas.max()
    extras = areas.size - 1

    # ——— decision ———
    # coherent if one blob covers ≥ main_frac_thresh of all mask pixels,
    # and there are at most max_extra_components others
    if (largest / total) >= main_frac_thresh and extras <= max_extra_components:
        return True
    else:
        return False

In [9]:
styled_images

{'/home/miguel/GI/1.5 - Synthetic Data Generation/diffuse-gen/diffuse-gen/guided_diffusion/segmented-images-roberta-hq-masks/FD-032/masked-images/FD-027-slice-07-image.png': array([[[[  8,   8,   8,   8],
          [ 13,  13,  13,  10],
          [ 12,  12,  12,   9],
          ...,
          [ 11,  11,  11,  10],
          [  9,   9,   9,  11],
          [  7,   7,   7,  11]],
 
         [[ 10,  10,  10,   7],
          [ 13,  13,  13,  11],
          [ 14,  14,  14,  10],
          ...,
          [ 12,  12,  12,  10],
          [ 11,  11,  11,   9],
          [  8,   8,   8,  11]],
 
         [[ 12,  12,  12,   8],
          [ 14,  14,  14,   8],
          [ 13,  13,  13,   8],
          ...,
          [ 13,  13,  13,   9],
          [ 13,  13,  13,  10],
          [  9,   9,   9,   9]],
 
         ...,
 
         [[ 11,  11,  11,   8],
          [ 15,  15,  15,  11],
          [ 14,  14,  14,  11],
          ...,
          [ 13,  13,  13,   8],
          [ 13,  13,  13,   8],
      

In [18]:

valid_mask_images_per_cv_subject = {}
subject_counts_per_cv_subject = {}
HIGH_SSIM_THRESHOLD = 0.15

for sample_index in range(len(paths)):
    styled_img_path = paths[sample_index]["styled_img_path"]

    with open(styled_img_path, "rb") as f:
        styled_images = pickle.load(f)

    filename_images = {}
    filename_masks = {}
    masked_images = os.listdir(masked_images_folder_path)

    for filename in masked_images:
        if not filename.endswith('.png') and not filename.endswith('.jpg'):
            continue
            
        img_path = os.path.join(masked_images_folder_path, filename)
        mask_path = os.path.join(masked_images_folder_path, filename.replace('-image', '-mask'))

        image = Image.open(img_path)
        mask = Image.open(mask_path)

        filename_images[filename] = image
        filename_masks[filename] = mask

    valid_mask_images = []
    valid_mask_subjects = []

    total_images = 0
    for key, styled_list in tqdm(styled_images.items()):
        for styled_image in tqdm(styled_list):
            total_images+=1 
            # Split synthetic image and mask
            synthetic_img = styled_image[:, :, :3].astype(np.uint8)
            synthetic_gray = cv2.cvtColor(np.asarray(synthetic_img), cv2.COLOR_RGB2GRAY)

            synthetic_mask = styled_image[:, :, 3].astype(np.uint8)
            synthetic_mask_normalized = styled_image[:, :, 3] / 255.0

            filename = key.split('/')[-1]  # Extract filename

            original_image = filename_images.get(filename, None)
            if original_image is None:
                print(f"Warning: No original image found for {filename}. Skipping.")
                continue

            original_mask = filename_masks.get(filename, None)
            if original_mask is None:
                print(f"Warning: No original mask found for {filename}. Skipping.")
                continue
            
            ssim_value = ssim(
                np.array(original_image),
                synthetic_gray,
                full=True
            )[0]


            mask_bool = synthetic_mask > 200
            percent_mask_highlighted = np.sum(mask_bool) / mask_bool.size 
            if percent_mask_highlighted < 0.5 and is_cohesive_mask(synthetic_mask_normalized, thresh_method='otsu', min_area=50, main_frac_thresh=0.6, max_extra_components=1):
                valid_mask_images.append(filename)
                valid_mask_subjects.append('-'.join(key.split('/')[-1].split('-')[:2]))
    print('CV subject:', paths[sample_index]['cv_subject'], 'has', len(valid_mask_images), 'valid mask images (total images:', total_images, ')')
    valid_mask_images_per_cv_subject[paths[sample_index]['cv_subject']] = valid_mask_images
    subject_counts_per_cv_subject[paths[sample_index]['cv_subject']] = pd.Series(valid_mask_subjects).value_counts()

100%|██████████| 20/20 [00:01<00:00, 12.95it/s]


CV subject: FD-027 has 49 valid mask images (total images: 400 )


100%|██████████| 28/28 [00:01<00:00, 15.06it/s]


CV subject: FD-029 has 84 valid mask images (total images: 560 )


100%|██████████| 43/43 [00:03<00:00, 13.41it/s]


CV subject: FD-031 has 136 valid mask images (total images: 860 )


100%|██████████| 167/167 [00:05<00:00, 28.47it/s]

CV subject: FD-032 has 1357 valid mask images (total images: 1460 )


In [8]:
full_data_path = '/home/miguel/GI/1.5 - Synthetic Data Generation/diffuse-gen/diffuse-gen/guided_diffusion/segmented-images-roberta/'
                  
export_path = '/home/miguel/GI/1.5 - Synthetic Data Generation/diffuse-gen/diffuse-gen/guided_diffusion/segmented-images-roberta-hq-masks/'
os.makedirs(export_path, exist_ok=True)

In [15]:
bounding_boxes_json_path

'/home/miguel/GI/1.5 - Synthetic Data Generation/diffuse-gen/diffuse-gen/guided_diffusion/segmented-images-roberta/FD-027/bounding_boxes.json'

In [21]:
for cv_subject in valid_mask_images_per_cv_subject:
    image_paths = valid_mask_images_per_cv_subject[cv_subject]
    cv_subject_export_path = os.path.join(export_path, cv_subject)
    os.makedirs(cv_subject_export_path, exist_ok=True)

    cv_subject_folder_full_data = os.path.join(full_data_path, cv_subject)
    bounding_boxes_json_path = os.path.join(cv_subject_folder_full_data, 'bounding-boxes.json')

    bounding_boxes = json.load(open(bounding_boxes_json_path, 'r'))

    for image_path in image_paths:
        del bounding_boxes[image_path]

    with open(os.path.join(cv_subject_export_path, f'bounding_boxes.json'), 'w') as f:
        json.dump(bounding_boxes, f, indent=4)
    
    original_mask_path = os.path.join(cv_subject_folder_full_data, 'masks')
    original_mask_images_path = os.path.join(cv_subject_folder_full_data, 'masked-images')

    export_mask_path = os.path.join(cv_subject_export_path, 'masks')
    export_mask_images_path = os.path.join(cv_subject_export_path, 'masked-images')
    
    os.makedirs(export_mask_path, exist_ok=True)
    os.makedirs(export_mask_images_path, exist_ok=True)

    for image_path in image_paths:
        shutil.copy(
            os.path.join(original_mask_images_path, image_path),
            os.path.join(export_mask_images_path, image_path)
        )
        shutil.copy(
            os.path.join(original_mask_path, image_path),
            os.path.join(export_mask_path, image_path)
        )
        

In [ ]:
'/home/miguel/GI/1.5 - Synthetic Data Generation/diffuse-gen/diffuse-gen/guided_diffusion/segmented-images-roberta/FD-027/masked-images/FD-029-slice-28-image.png'
'/home/miguel/GI/1.5 - Synthetic Data Generation/diffuse-gen/diffuse-gen/guided_diffusion/segmented-images-roberta-hq-masks/FD-027/masked-images/FD-029-slice-28-image.png'

'/home/miguel/GI/1.5 - Synthetic Data Generation/diffuse-gen/diffuse-gen/guided_diffusion/segmented-images-roberta/FD-027/masked-images/FD-029-slice-28-image.png'

In [22]:
total_images = 0
for cv_subject in valid_mask_images_per_cv_subject:
    num_images = len(valid_mask_images_per_cv_subject[cv_subject])
    total_images += num_images

In [23]:
total_images

258

In [10]:
valid_mask_images_per_cv_subject

{'FD-027': ['FD-029-slice-28-image.png',
  'FD-029-slice-33-image.png',
  'FD-030-slice-39-image.png',
  'FD-030-slice-41-image.png',
  'FD-030-slice-45-image.png',
  'FD-030-slice-47-image.png',
  'FD-030-slice-51-image.png',
  'FD-030-slice-60-image.png',
  'FD-030-slice-66-image.png',
  'FD-031-slice-17-image.png',
  'FD-031-slice-29-image.png',
  'FD-031-slice-37-image.png',
  'FD-031-slice-44-image.png',
  'FD-031-slice-47-image.png',
  'FD-032-slice-23-image.png',
  'FD-032-slice-25-image.png',
  'FD-032-slice-26-image.png',
  'FD-032-slice-30-image.png',
  'FD-032-slice-33-image.png',
  'FD-032-slice-57-image.png'],
 'FD-029': ['FD-027-slice-10-image.png',
  'FD-027-slice-16-image.png',
  'FD-027-slice-18-image.png',
  'FD-027-slice-20-image.png',
  'FD-027-slice-31-image.png',
  'FD-027-slice-34-image.png',
  'FD-027-slice-38-image.png',
  'FD-027-slice-42-image.png',
  'FD-030-slice-29-image.png',
  'FD-030-slice-33-image.png',
  'FD-030-slice-48-image.png',
  'FD-030-slice-53

100%|██████████| 183/183 [00:01<00:00, 145.71it/s]


In [10]:
len(high_ssim_images) / len(styled_images) 

0.07103825136612021